# 지도 학습 기반 형태소 분석
## KoNLPy 형태소 분석기 활용

- Mecab (a.k.a 은전한닢)을 활용한 형태소 분석이 가장 대중적인 Tokenization 방법

In [8]:
from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs('아버지가방에들어가신다')

['아버지', '가', '방', '에', '들어가', '신다']

- `morphs` function을 이용하면 형태소 단위의 **tokenization**이 가능하고,
- `pos` function을 이용하면 각 형태소의 **POS (Part-Of-Speech)** Tag를 형태소와 함께 출력 가능

In [9]:
tokenizer.pos('아버지가방에들어가신다')

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EC')]

- 교재에서는 KoNLPy에 내장되어 있는 다양한 형태소 분석기를 유연하게 사용하기 위해 `get_tokenizer` 함수를 정의하였으나,
- `Mecab`을 사용할 수 있는 환경이라면 굳이 다른 형태소 분석기를 사용할 이유가 없는 것으로 알고 있음

> +) **조건문**으로 인한 분기를 최대한 피하고 의식적으로 **Key access**를 지향하자!
>
> _github repository 내 [slide](https://github.com/nlp-kkmas/korean-embedding-study/blob/master/treasure/Clean%20code%20for%20AI%2C%20ML.pdf) 참고_

In [ ]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

def get_tokenizer(name):
    key = {'komoran': Komoran(),
           'okt': Okt(),
           'mecab': Mecab(),
           'hannanum': Hannanum(),
           'kkma': Kkma()}
    
    if name not in key:
        name = 'mecab'
    
    return key[name]        

기정의한 `get_tokenizer` function을 이용해 형태소 분석기를 생성하는 예시

In [ ]:
tokenizer = get_tokenizer('komoran')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

In [ ]:
tokenizer = get_tokenizer('bullshit')
print(type(tokenizer))
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

> KoNLPy 내 형태소 분석기 간 비교는 다음 글을 참조하자 ! : [누가누가 잘하나! 대화체와 합이 잘 맞는 Tokenizer를 찾아보자!](https://blog.pingpong.us/tokenizer/)

## Khaiii 형태소 분석기 활용

- MacOS, linux 환경에서는 카카오가 제작한 `Khaiii` 형태소 분석기를 활용 가능
- Mecab 등과 달리 자연어 처리에 적용할 경우, **원문을 복원**하는 과정이 추가적으로 필요

In [ ]:
from khaiii import KhaiiiApi

tokenizer = KhaiiiApi()
data = tokenizer.analyze("아버지가방에들어가신다")

tokens = []
clean_tokens = []

for word in data:
    tokens.extend([m.lex for m in word.morphs])
    clean_tokens.extend([(m.lex, m.tag) for m in word.morphs])

print(f'Tokens with morpheme information: {tokens}')
print(f'Tokens without morpheme information: {clean_tokens}')

## 사용자 사전 추가

- 진행하는 프로젝트에 따라 기학습된 규칙을 기반으로 형태소 분석을 수행하는 KoNLPy 형태소 분석기들에 **형태소로 분리되지 않아야 하는** 사용자 정의 단어를 추가해줄 필요가 있을 수 있음
- 이럴 때 **사용자 사전**을 정의해 각 형태소 분석기에 추가해줄 수 있음

In [1]:
# bad case:
from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs('가우스전자 텔레비전 정말 좋네요')

['가우스', '전자', '텔레비전', '정말', '좋', '네요']

- 표층형/0/0/0/품사태그/의미분류/종성유무/읽기/타입/첫번째품사/마지막품사/표현 ([link](https://kugancity.tistory.com/entry/mecab%EC%97%90-%EC%82%AC%EC%9A%A9%EC%9E%90%EC%82%AC%EC%A0%84%EA%B8%B0%EB%B6%84%EC%84%9D-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0))

> 가우스전자,,,,NNP,**,T,가우스전자,**,**,**,**,**

> bash preprocess.sh [mecab-user-dic](https://github.com/ratsgo/embedding/blob/master/preprocess.sh#L195) 참조

# 비지도 학습 기반 형태소 분석

## soynlp
- 형태소 분석, 품사 판별 등을 지원하는 한국어 자연어 처리 패키지
- 하나의 문장/문서에서보다는 어느 정도 규모가 있으면서 동질적인 문서 집합에서 잘 작동
- soynlp의 학습 지표
    1. **Cohesion Probability**: 주어진 문자열이 유기적으로 연결돼 함께 자주 나타나는가?
    2. **Branching Entropy**: 해당 단어 앞뒤로 다양한 조사, 어미 혹은 다른 단어가 등장하는가?
- soynlp와 같은 비지도 학습 기반의 library는 corpus를 바탕으로 한 학습 이후 사용 가능

### soynlp 학습 코드

In [4]:
from soynlp.word import WordExtractor

corpus_fname = 'data/processed_ratings.txt'
model_fname = 'data/soyword.model'

sentences = [sent.strip() for sent in open(corpus_fname, 'r').readlines()]
word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05,
                               min_right_branching_entropy=0.0)
word_extractor.train(sentences)
word_extractor.save(model_fname)

training was done. used memory 0.745 Gbse memory 0.541 Gb


### 학습된 soynlp 모델의 점수를 이용한 형태소 분석

In [4]:
import math
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

model_fname = 'data/soyword.model'
word_extractor = WordExtractor(min_frequency=100,
                               min_cohesion_forward=0.05,
                               min_right_branching_entropy=0.0)
word_extractor.load(model_fname)

scores = word_extractor.word_scores()
scores = {key:(scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) for key in scores.keys()}

tokenizer = LTokenizer(scores=scores)
tokens = tokenizer.tokenize("애비는 종이었다")
print(tokens)

all cohesion probabilities was computed. # words = 6130
all branching entropies was computed # words = 123575
all accessor variety was computed # words = 123575
['애비는종이었다']


## Google SentencePiece
- Google에서 공개한 BPE (Byte Pair Encoding) 기반의 Tokenizer
- Corpus에서 가장 많이 등장한 문자열을 병합해 압축하여 사용
- Goal: 원하는 Vocab size를 갖출 때 까지 반복적으로 고빈도 문자열을 병합해 Vocab에 추가 !
    - 띄어쓰기를 기준으로 나눈 어절에 Sub-word가 포함되어 있는 경우 **최장 길이 일치**를 기준으로 Subword를 분리
    - Subword 탐색을 반복한 후, Vocab에 없는 단어를 만나는 경우 **OOV** (Out Of Vocabulary) 처리
- **BERT**를 비롯한 다양한 신모델들은 BPE 기반의 Tokenizer를 사용
    - 사실상 **Transformer**의 등장 이후, 영어권에서는 **BPE Tokenization**이 **최선책**으로 등극

### SentencePiece 학습 코드 (don't run it)

In [ ]:
import sentencepiece as spm

train = '--input=data/processed_wiki_ko.txt --model_prefix=sentpiece --vocab_size=32000 --model_type=bpe --character_coverage=0.9995'
spm.SentencePieceTrainer.Train(train)

### BPE 기반의 BERT Tokenizer (using. special tokens)

In [7]:
from bert.tokenization import FullTokenizer

vocab_fname = 'data/bert.vocab'
tokenizer = FullTokenizer(vocab_file=vocab_fname, do_lower_case=False)

print(tokenizer.tokenize('집에좀 가자'))

ImportError: No module named 'bert'

### 띄어쓰기 교정 모델 학습 코드

In [ ]:
from soyspacing.countbase import CountSpace

corpus_fname = 'data/processed_ratings.txt'
model_fname = 'data/space-correct.model'

model = CountSpace()
model.train(corpus_fname)
model.save_model(model_fname, json_format=False)

### 띄어쓰기 교정 모델 추론

In [6]:
from soyspacing.countbase import CountSpace

model_fname = 'data/space-correct.model'
model = CountSpace()
model.load_model(model_fname, json_format=False)
model.correct('어릴때보고 지금다시봐도 재밌어요')

('어릴때 보고 지금 다시봐도 재밌어요', [0, 0, 1, 0, 1, 0, 1, 0, None, 0, 1, 0, 0, 0, 1])

## 참조. MaxScoreTokenizer
띄어쓰기가 제대로 지켜지지 않은 데이터라면, 문장의 띄어쓰기 기준으로 나뉘어진 단위가 L + [R] 구조라 가정할 수 없습니다. 하지만 사람은 띄어쓰기가 지켜지지 않은 문장에서 익숙한 단어부터 눈에 들어옵니다. 이 과정을 모델로 옮긴 MaxScoreTokenizer 역시 단어 점수를 이용합니다.

```
from soynlp.tokenizer import MaxScoreTokenizer

scores = {'파스': 0.3, '파스타': 0.7, '좋아요': 0.2, '좋아':0.5}
tokenizer = MaxScoreTokenizer(scores=scores)

print(tokenizer.tokenize('난파스타가좋아요'))
# ['난', '파스타', '가', '좋아', '요']

print(tokenizer.tokenize('난파스타가 좋아요', flatten=False))
# [[('난', 0, 1, 0.0, 1), ('파스타', 1, 4, 0.7, 3),  ('가', 4, 5, 0.0, 1)],
#  [('좋아', 0, 2, 0.5, 2), ('요', 2, 3, 0.0, 1)]]
```

MaxScoreTokenizer 역시 WordExtractor 의 결과를 이용하실 때에는 위의 예시처럼 적절히 scores 를 만들어 사용합니다. 이미 알려진 단어 사전이 있다면 이 단어들은 다른 어떤 단어보다도 더 큰 점수를 부여하면 그 단어는 토크나이저가 하나의 단어로 잘라냅니다.

> 구어체에 띄어쓰기를 적용하고 싶을 경우, [Ping-pong's chatspace](https://github.com/pingpong-ai/chatspace) 참조 !